In [42]:
# Importing necessary librariers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy

import re
import gensim
import pyLDAvis
import warnings
import nltk

%matplotlib inline

In [63]:
from spacy import *

In [43]:
# Importing the dataset

dataset = pd.read_csv('C:/Users/vaibh/Desktop/360 Digitmg/Topic Modeling and Text Summarization/Data.csv')

In [44]:
# Checking first five rows of the dataset
dataset.head()

,tweet_id,sentiment,text,tweet_created,tweet_location,user_timezone
0,1,neutral,What @dhepburn said.,24/02/2015 11:35,NaN,Eastern Time (US & Canada)
1,2,positive,plus you've added commercials to the experienc...,24/02/2015 11:15,NaN,Pacific Time (US & Canada)
2,3,neutral,I didn't today... Must mean I need to take ano...,24/02/2015 11:15,Lets Play,Central Time (US & Canada)
3,4,negative,"it's really aggressive to blast obnoxious ""ent...",24/02/2015 11:15,NaN,Pacific Time (US & Canada)
4,5,negative,and it's a really big bad thing about it,24/02/2015 11:14,NaN,Pacific Time (US & Canada)


In [45]:
# Checking last five rows of the dataset
dataset.tail()

,tweet_id,sentiment,text,tweet_created,tweet_location,user_timezone
14635,14636,positive,thank you we got on a different flight to Chic...,22/02/2015 12:01,NaN,NaN
14636,14637,negative,leaving over 20 minutes Late Flight. No warnin...,22/02/2015 11:59,Texas,NaN
14637,14638,neutral,Please bring American Airlines to #BlackBerry10,22/02/2015 11:59,"Nigeria,lagos",NaN
14638,14639,negative,"you have my money, you change my flight, and d...",22/02/2015 11:59,New Jersey,Eastern Time (US & Canada)
14639,14640,neutral,we have 8 ppl so we need 2 know how many seats...,22/02/2015 11:58,"dallas, TX",NaN


In [46]:
# Information of the dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        14640 non-null  int64 
 1   sentiment       14640 non-null  object
 2   text            14640 non-null  object
 3   tweet_created   14640 non-null  object
 4   tweet_location  9907 non-null   object
 5   user_timezone   9820 non-null   object
dtypes: int64(1), object(5)
memory usage: 686.4+ KB


#### There are Nan values in tweet_location and user_timezone, but is irrelevant to the study
#### We will be focusing on the text column of the dataset

In [47]:
# Extracting text feature
data = dataset.text
data.head()

0                                 What @dhepburn said.
1    plus you've added commercials to the experienc...
2    I didn't today... Must mean I need to take ano...
3    it's really aggressive to blast obnoxious "ent...
4             and it's a really big bad thing about it
Name: text, dtype: object

In [48]:
# Cleaning the data
warnings.filterwarnings(action='ignore',category=DeprecationWarning)

handle = '@\w+'
Link = r'\S*https?:\S*'
Special_cha = '&lt;|&lt;|&amp;|#'

def clean_data(data):
    data = re.sub(handle,'',data)
    data = re.sub(Link,'',data)
    data = re.sub(Special_cha,'',data)
    return data

In [49]:
# Applying function clean_data 
text = data.apply(clean_data)
text.head()

0                                          What  said.
1    plus you've added commercials to the experienc...
2    I didn't today... Must mean I need to take ano...
3    it's really aggressive to blast obnoxious "ent...
4             and it's a really big bad thing about it
Name: text, dtype: object

In [50]:
def clean_num(data):
    data = re.sub(r'\d+','',data)
    return data

In [51]:
text = text.apply(clean_num)
text.head(10)

0                                          What  said.
1    plus you've added commercials to the experienc...
2    I didn't today... Must mean I need to take ano...
3    it's really aggressive to blast obnoxious "ent...
4             and it's a really big bad thing about it
5    seriously would pay $ a flight for seats that ...
6    yes, nearly every time I fly VX this __ar worm...
7    Really missed a prime opportunity for Men With...
8                     Well, I didn't_but NOW I DO! :-D
9    it was amazing, and arrived an hour early. You...
Name: text, dtype: object

In [52]:
# Removing stopwords
from nltk.corpus import stopwords
stopword = stopwords.words('english')

new_text = []
for i in text:
    temp = []
    sp_text = i.lower().split()
    for j in sp_text:
        if j not in stopword:
            temp.append(j)
    new_temp = " ".join(temp)
    new_text.append(new_temp)

In [53]:
new_text

['said.',
 'plus added commercials experience... tacky.',
 'today... must mean need take another trip!',
 'really aggressive blast obnoxious "entertainment" guests\' faces little recourse',
 'really big bad thing',
 'seriously would pay $ flight seats playing. really bad thing flying va',
 'yes, nearly every time fly vx __ar worm_ won__ go away :)',
 'really missed prime opportunity men without hats parody, there.',
 "well, didn't_but do! :-d",
 'amazing, arrived hour early. good me.',
 'know suicide second leading cause death among teens -',
 'pretty graphics. much better minimal iconography. :d',
 'great deal! already thinking nd trip even gone st trip yet! ;p',
 "i'm flying fabulous seductive skies again! u take stress away travel",
 'thanks!',
 'sfo-pdx schedule still mia.',
 "excited first cross country flight lax mco i've heard nothing great things virgin america. daystogo",
 'flew nyc sfo last week fully sit seat due two large gentleman either side me. help!',
 '___ flying . ___

In [54]:
# Removing punctuation 

from gensim.parsing.preprocessing import strip_punctuation

new_text = [strip_punctuation(i) for i in new_text]

In [55]:
new_text

['said ',
 'plus added commercials experience  tacky ',
 'today  must mean need take another trip ',
 'really aggressive blast obnoxious  entertainment  guests  faces little recourse',
 'really big bad thing',
 'seriously would pay   flight seats playing  really bad thing flying va',
 'yes  nearly every time fly vx  ar worm  won  go away  ',
 'really missed prime opportunity men without hats parody  there ',
 'well  didn t but do   d',
 'amazing  arrived hour early  good me ',
 'know suicide second leading cause death among teens  ',
 'pretty graphics  much better minimal iconography   d',
 'great deal  already thinking nd trip even gone st trip yet   p',
 'i m flying fabulous seductive skies again  u take stress away travel',
 'thanks ',
 'sfo pdx schedule still mia ',
 'excited first cross country flight lax mco i ve heard nothing great things virgin america  daystogo',
 'flew nyc sfo last week fully sit seat due two large gentleman either side me  help ',
 '  flying    ',
 'know wou

In [56]:
new1_text = []
for i in new_text:
    temp = []
    sp_text = i.lower().split()
    for j in sp_text:
        if j not in stopword:
            temp.append(j)
    new_temp = " ".join(temp)
    new1_text.append(new_temp)

In [57]:
# from nltk.stem import WordNetLemmatizer
# Lemma = WordNetLemmatizer()

# new_text = [Lemma.lemmatize(i) for i in new1_text]

In [64]:
def Lemmatization(texts,allowed_postags= ['NOUN','ADJ','VERB','ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in text:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final =' '.join(new_text)
        texts_out.append(final)
    return (texts_out)

In [65]:
lem_text = Lemmatization(new1_text)
lem_text

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
new2_text = []
for i in new_text:
    temp = []
    sp_text = i.lower().split()
    for j in sp_text:
        if len(j) not in [0,1,2]:
            temp.append(j)
    new_temp = " ".join(temp)
    new2_text.append(new_temp)

In [ ]:
from gensim import corpora

new2_text = [nltk.tokenize.word_tokenize(i) for i in new2_text]
dictionary = corpora.Dictionary(new2_text)
corpus = [dictionary.doc2bow(i) for i in new2_text]

In [ ]:
corpus

In [ ]:
from gensim.models.ldamodel import LdaModel

NUM_TOPICS = 5
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=10)

In [ ]:
ldamodel.print_topics(num_words = 5)